In [1]:
!nvidia-smi

Tue Dec 19 01:04:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from pathlib import Path

HOME = os.getcwd()

# The name for the project
PROJECT_NAME = f"Tracker"

# The path for the project folder
PROJECT_DIR = Path(f"./{PROJECT_NAME}")

# Create the project directory if it does not already exist
PROJECT_DIR.mkdir(parents=True, exist_ok=True)

%cd {PROJECT_DIR}

# Default GOOGLE DRIVE
GDRIVE_VID_PATH = f"/content/drive/MyDrive/detect_human/Samples"

/content/Tracker


In [3]:
!gdown "https://drive.google.com/uc?id=1D8HO3ina1HXOIrZpyJ4gpRgBPZlssmmK&confirm=t"
!gdown "https://drive.google.com/uc?id=1yqUPkFXFF_5AOi6QaMxyonGazA-Fe9tN&confirm=t"
!gdown "https://drive.google.com/uc?id=1LZ_uasbiu7nZbo8Mef5zAxJjfxyWrCZW&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1D8HO3ina1HXOIrZpyJ4gpRgBPZlssmmK&confirm=t
To: /content/Tracker/++_01_Cam01_20171223132726_300__K1S2.mp4
100% 131M/131M [00:01<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yqUPkFXFF_5AOi6QaMxyonGazA-Fe9tN&confirm=t
To: /content/Tracker/best_yolov8m_200.pt
100% 52.1M/52.1M [00:00<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LZ_uasbiu7nZbo8Mef5zAxJjfxyWrCZW&confirm=t
To: /content/Tracker/best_yolov8m_59000.pt
100% 52.0M/52.0M [00:00<00:00, 97.8MB/s]


In [4]:
SOURCE_VIDEO_PATH = f"{HOME}/{PROJECT_NAME}/test.mp4"

In [5]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/166.8 GB disk)


In [ ]:
 %cd {HOME}
 !git clone https://github.com/ifzhang/ByteTrack.git
 !cd ByteTrack && pip install -q -r requirements.txt
 !cd ByteTrack && python setup.py -q develop
 !pip install -q cython_bbox
 !pip install -q onemetric
 !pip install loguru lap

In [7]:
import sys
sys.path.append(f"{HOME}/ByteTrack")

In [8]:
import yolox
print("yolox.__version__:{}".format(yolox.__version__))

yolox.__version__:0.1.0


In [9]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

@dataclass(frozen=True)
class BYTETrackerArgs:
  track_thresh: float = 0.25
  track_buffer: int = 90
  # match_trhesh: float = 0.8
  aspect_ratio_thresh:float = 3.0
  min_box_area: float = 1.0
  mot20: bool = False

In [10]:
!pip install supervision==0.1.0

display.clear_output()

import supervision
print("supervision.__version__:{}".format(supervision.__version__))

supervision.__version__:0.1.0


In [11]:
MODEL = "best_yolov8m_59000.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(f"{HOME}/{PROJECT_NAME}/{MODEL}")
model.fuse()

In [13]:
CLASS_NAMES_DICT = model.names
CLASS_NAMES_DICT

{0: 'human', 1: 'head'}

In [ ]:
!yolo task=detect mode=predict model=/content/Tracker/best_yolov8m_200.pt conf=0.25 source=/content/Tracker/frame_0001.jpeg

In [ ]:
!yolo task=detect mode=predict model=/content/Tracker/best_yolov8m_59000.pt conf=0.25 source={SOURCE_VIDEO_PATH}

In [24]:
!cp /content/runs/detect/predict/test.avi /content/drive/MyDrive/Original/result/

In [14]:
from typing import Generator

import matplotlib.pyplot as plt
import numpy as np

import cv2

%matplotlib inline

def generate_frames(video_file: str) -> Generator[np.ndarray, None, None]:
  video = cv2.VideoCapture(video_file)

  while video.isOpened():
    success, frame = video.read()

    if not success:
      break

    yield frame

  video.release()

def plot_image(image: np.ndarray, size: int = 12) -> None:
  plt.figure(figsize=(size, size))
  plt.imshow(image[..., ::-1])
  plt.show()


In [15]:
SOURCE_VIDEO_PATH

'/content/Tracker/test.mp4'

In [23]:
iterator = iter(generate_frames(video_file=SOURCE_VIDEO_PATH))
frame = next(iterator)
results = model(frame)[0]


WARNING ⚠️ NMS time limit 0.550s exceeded
0: 384x640 10 humans, 275.4ms
Speed: 13.0ms preprocess, 275.4ms inference, 1050.1ms postprocess per image at shape (1, 3, 384, 640)


In [16]:
from supervision.draw.color import ColorPalette, Color
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [17]:
from typing import Tuple

In [18]:
# green
HEAD_HEX = "#00D4BB"
HEAD_COLOR = Color.from_hex(HEAD_HEX)

# yellow
PERSON_HEX = "#FFFF00"
PERSON_COLOR = Color.from_hex(PERSON_HEX)

COLORS = [
    HEAD_COLOR,
    PERSON_COLOR
]
THICKNESS = 1

In [19]:
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
box_annotator = BoxAnnotator(color=HEAD_COLOR, thickness=4, text_thickness=4, text_scale=2)

In [24]:
detections = Detections(
  xyxy=results.boxes.xyxy.cpu().numpy(),
  confidence=results.boxes.conf.cpu().numpy(),
  class_id=results.boxes.cls.cpu().numpy().astype(int)
)

In [25]:
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
video_info

VideoInfo(width=1280, height=720, fps=25, total_frames=7487)

In [26]:
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]

In [27]:
TARGET_VIDEO_PATH = f"{HOME}/{PROJECT_NAME}/result.mp4"

In [28]:
from tqdm.notebook import tqdm
from typing import List

In [29]:
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
from supervision.draw.color import ColorPalette, Color
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

from tqdm.notebook import tqdm

@dataclass(frozen=True)
class BYTETrackerArgs:
  track_thresh: float = 0.25
  track_buffer: int = 900
  match_thresh: float = 0.8
  aspect_ratio_thresh:float = 3.0
  min_box_area: float = 1.0
  mot20: bool = False

byte_tracker = BYTETracker(BYTETrackerArgs())

generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
box_annotator = BoxAnnotator(color=HEAD_COLOR, thickness=2, text_thickness=1, text_scale=1)

video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
  for frame in tqdm(generator, total=video_info.total_frames):
    results = model(frame)[0]

    detections = Detections(
      xyxy=results.boxes.xyxy.cpu().numpy(),
      confidence=results.boxes.conf.cpu().numpy(),
      class_id=results.boxes.cls.cpu().numpy().astype(int)
    )

    tracks = byte_tracker.update(
        output_results = detections2boxes(detections=detections),
        img_info=frame.shape,
        img_size=frame.shape
    )

    tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)

    detections.tracker_id = np.array(tracker_id)

    labels = [
      f"#{tracker_id}"
      for _, confidence, class_id, tracker_id
      in detections
    ]

    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

    sink.write_frame(frame)

In [33]:
!cp /content/Tracker/result.mp4 /content/drive/MyDrive/Original/result/

In [ ]:
from google.colab import files
files.download('/content/Tracker/result.mp4')